# Portfolio Analysis Notebook

Interactive analysis of your Investment Platform portfolio.

**Prerequisites:**
- Investment Platform API running at http://localhost:5000
- Required packages: requests, pandas, plotly

In [ ]:
# Setup and imports
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from IPython.display import display, HTML

# API Configuration
BASE_URL = 'http://localhost:5000/api'
USER_ID = 'default'

print(f"API Base URL: {BASE_URL}")
print(f"User ID: {USER_ID}")

In [ ]:
# Helper functions

def get_portfolio():
    """Fetch current portfolio settings."""
    response = requests.get(f'{BASE_URL}/portfolio/settings', params={'user_id': USER_ID})
    response.raise_for_status()
    return response.json()

def get_holdings():
    """Fetch current holdings."""
    response = requests.get(f'{BASE_URL}/holdings', params={'user_id': USER_ID})
    response.raise_for_status()
    return response.json()

def get_trades(limit=100):
    """Fetch trade history."""
    response = requests.get(f'{BASE_URL}/trades', params={'user_id': USER_ID, 'limit': limit})
    response.raise_for_status()
    return response.json()

def get_market_prices(symbols):
    """Fetch current market prices."""
    response = requests.get(f'{BASE_URL}/market/prices', params={'symbols': ','.join(symbols)})
    response.raise_for_status()
    return response.json()

# Test API connection
try:
    health = requests.get(f'{BASE_URL}/health').json()
    print(f"API Status: {health['status']}")
except Exception as e:
    print(f"ERROR: Cannot connect to API - {e}")
    print("Make sure the Investment Platform is running!")

## 1. Portfolio Overview

In [ ]:
# Fetch portfolio data
portfolio = get_portfolio()
holdings_data = get_holdings()

# Display portfolio summary
print("=" * 50)
print("PORTFOLIO SUMMARY")
print("=" * 50)
print(f"Initial Value:    ${portfolio['initial_value']:,.2f}")
print(f"Current Cash:     ${portfolio['current_cash']:,.2f}")
print(f"Strategy:         {portfolio['current_strategy'].title()}")
print(f"Realized Gains:   ${portfolio['realized_gains']:,.2f}")
print(f"Estimated Tax:    ${portfolio['realized_gains'] * 0.37:,.2f} (@ 37%)")
print("=" * 50)

In [ ]:
# Convert holdings to DataFrame
if holdings_data:
    df_holdings = pd.DataFrame(holdings_data)
    
    # Calculate additional metrics
    df_holdings['value'] = df_holdings['quantity'] * df_holdings['current_price']
    df_holdings['cost_basis'] = df_holdings['quantity'] * df_holdings['avg_cost']
    df_holdings['gain_loss'] = df_holdings['value'] - df_holdings['cost_basis']
    df_holdings['gain_loss_pct'] = (df_holdings['gain_loss'] / df_holdings['cost_basis']) * 100
    
    # Display holdings table
    display_cols = ['symbol', 'name', 'sector', 'quantity', 'avg_cost', 'current_price', 'value', 'gain_loss', 'gain_loss_pct']
    display(df_holdings[display_cols].style.format({
        'quantity': '{:.0f}',
        'avg_cost': '${:.2f}',
        'current_price': '${:.2f}',
        'value': '${:,.2f}',
        'gain_loss': '${:,.2f}',
        'gain_loss_pct': '{:.2f}%'
    }).applymap(lambda x: 'color: green' if isinstance(x, str) and not x.startswith('-') else 'color: red', 
                subset=['gain_loss', 'gain_loss_pct']))
    
    # Summary stats
    total_invested = df_holdings['value'].sum()
    total_cost = df_holdings['cost_basis'].sum()
    total_gain = df_holdings['gain_loss'].sum()
    
    print(f"\nTotal Invested Value: ${total_invested:,.2f}")
    print(f"Total Cost Basis:     ${total_cost:,.2f}")
    print(f"Unrealized Gain/Loss: ${total_gain:,.2f} ({total_gain/total_cost*100:.2f}%)")
else:
    print("No holdings found. Portfolio is all cash.")

## 2. Holdings Visualization

In [ ]:
if holdings_data:
    # Pie chart - Allocation by Stock
    fig = px.pie(
        df_holdings, 
        values='value', 
        names='symbol',
        title='Portfolio Allocation by Stock',
        hole=0.4
    )
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.show()
else:
    print("No holdings to visualize.")

In [ ]:
if holdings_data:
    # Pie chart - Allocation by Sector
    sector_allocation = df_holdings.groupby('sector')['value'].sum().reset_index()
    
    fig = px.pie(
        sector_allocation, 
        values='value', 
        names='sector',
        title='Portfolio Allocation by Sector',
        hole=0.4
    )
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.show()

In [ ]:
if holdings_data:
    # Bar chart - Gain/Loss by Stock
    colors = ['green' if x >= 0 else 'red' for x in df_holdings['gain_loss']]
    
    fig = go.Figure(data=[
        go.Bar(
            x=df_holdings['symbol'],
            y=df_holdings['gain_loss'],
            marker_color=colors,
            text=[f"${x:,.0f}" for x in df_holdings['gain_loss']],
            textposition='outside'
        )
    ])
    
    fig.update_layout(
        title='Unrealized Gain/Loss by Stock',
        xaxis_title='Stock',
        yaxis_title='Gain/Loss ($)',
        showlegend=False
    )
    fig.show()

## 3. Trade History Analysis

In [ ]:
# Fetch trade history
trades = get_trades(limit=100)

if trades:
    df_trades = pd.DataFrame(trades)
    df_trades['timestamp'] = pd.to_datetime(df_trades['timestamp'])
    
    # Summary statistics
    buy_trades = df_trades[df_trades['type'] == 'buy']
    sell_trades = df_trades[df_trades['type'] == 'sell']
    
    print("=" * 50)
    print("TRADE HISTORY SUMMARY")
    print("=" * 50)
    print(f"Total Trades:     {len(df_trades)}")
    print(f"Buy Trades:       {len(buy_trades)} (${buy_trades['total'].sum():,.2f})")
    print(f"Sell Trades:      {len(sell_trades)} (${sell_trades['total'].sum():,.2f})")
    print(f"Total Fees Paid:  ${df_trades['fees'].sum():,.2f}")
    print("=" * 50)
    
    # Recent trades
    print("\nRecent Trades:")
    display(df_trades[['timestamp', 'type', 'symbol', 'quantity', 'price', 'total']].head(10))
else:
    print("No trade history found.")

In [ ]:
if trades:
    # Trade volume by stock
    trade_volume = df_trades.groupby(['symbol', 'type'])['total'].sum().unstack(fill_value=0)
    
    fig = go.Figure(data=[
        go.Bar(name='Buy', x=trade_volume.index, y=trade_volume.get('buy', 0), marker_color='green'),
        go.Bar(name='Sell', x=trade_volume.index, y=trade_volume.get('sell', 0), marker_color='red')
    ])
    
    fig.update_layout(
        title='Trade Volume by Stock',
        xaxis_title='Stock',
        yaxis_title='Total Volume ($)',
        barmode='group'
    )
    fig.show()

In [ ]:
if trades:
    # Trades over time
    df_trades['date'] = df_trades['timestamp'].dt.date
    daily_trades = df_trades.groupby(['date', 'type'])['total'].sum().unstack(fill_value=0)
    
    fig = go.Figure()
    
    if 'buy' in daily_trades.columns:
        fig.add_trace(go.Scatter(
            x=daily_trades.index, 
            y=daily_trades['buy'],
            mode='lines+markers',
            name='Buy Volume',
            line=dict(color='green')
        ))
    
    if 'sell' in daily_trades.columns:
        fig.add_trace(go.Scatter(
            x=daily_trades.index, 
            y=daily_trades['sell'],
            mode='lines+markers',
            name='Sell Volume',
            line=dict(color='red')
        ))
    
    fig.update_layout(
        title='Daily Trade Volume Over Time',
        xaxis_title='Date',
        yaxis_title='Volume ($)'
    )
    fig.show()

## 4. Performance Metrics

In [ ]:
# Calculate comprehensive performance metrics

portfolio = get_portfolio()
holdings_data = get_holdings()

# Total portfolio value
cash = portfolio['current_cash']
invested = sum(h['quantity'] * h.get('current_price', h['avg_cost']) for h in holdings_data) if holdings_data else 0
total_value = cash + invested
initial_value = portfolio['initial_value']

# Returns
total_return = total_value - initial_value
total_return_pct = (total_return / initial_value) * 100

# Unrealized gains
unrealized_gain = sum(
    (h.get('current_price', h['avg_cost']) - h['avg_cost']) * h['quantity'] 
    for h in holdings_data
) if holdings_data else 0

# Realized gains
realized_gain = portfolio['realized_gains']

# Tax liability
tax_liability = max(0, realized_gain * 0.37)

# Investment ratio
investment_ratio = (invested / total_value * 100) if total_value > 0 else 0

print("=" * 50)
print("PERFORMANCE METRICS")
print("=" * 50)
print(f"\nPortfolio Value")
print(f"  Cash:           ${cash:>15,.2f}")
print(f"  Invested:       ${invested:>15,.2f}")
print(f"  Total:          ${total_value:>15,.2f}")
print(f"\nReturns")
print(f"  Total Return:   ${total_return:>15,.2f} ({total_return_pct:+.2f}%)")
print(f"  Unrealized:     ${unrealized_gain:>15,.2f}")
print(f"  Realized:       ${realized_gain:>15,.2f}")
print(f"\nTax & Ratios")
print(f"  Est. Tax:       ${tax_liability:>15,.2f}")
print(f"  Investment %:   {investment_ratio:>15.1f}%")
print("=" * 50)

## 5. Interactive Widgets (Optional)

Run this cell to enable interactive exploration.

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import clear_output
    
    # Create widgets
    refresh_button = widgets.Button(description="Refresh Data")
    output = widgets.Output()
    
    def on_refresh_click(b):
        with output:
            clear_output(wait=True)
            portfolio = get_portfolio()
            holdings = get_holdings()
            
            print(f"Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"Cash: ${portfolio['current_cash']:,.2f}")
            print(f"Holdings: {len(holdings)} positions")
            
            if holdings:
                total = sum(h['quantity'] * h.get('current_price', h['avg_cost']) for h in holdings)
                print(f"Invested: ${total:,.2f}")
    
    refresh_button.on_click(on_refresh_click)
    
    display(widgets.VBox([refresh_button, output]))
    
except ImportError:
    print("ipywidgets not installed. Run: pip install ipywidgets")